# Cannibalization Prediction

In [1]:
# Step 1: Set environment
# We need to set environment's correct values if we want to load project modules. 
import sys, os

# It's important load the PROJECT_ROOT path. We need to replace PROJECT_ROOT with correct folder
# Module is placed in api folder inside project root folder
# PROJECT_ROOT = '/Users/44371/Documents/Cases/V7FC/bain-vantage'
PROJECT_ROOT = '/home/jacrisol/github/bain-vantage'
MODULE_FULL_PATH = os.path.join(PROJECT_ROOT, 'api')
sys.path.insert(1, MODULE_FULL_PATH)


from api.settings import env, config

# config file sample can be foun at PROJECT_ROOT/api/config.env.sample. All keys and passwods are placed there.
# in this step, we load this file to get all carto api keys needed.
# env.read_env('/Users/44371/Documents/Cases/V7FC/bain-vantage/api/notebooks.env')
env.read_env('/home/jacrisol/github/bain-vantage/api/notebooks.env')
with env.prefixed("BAIN_VANT_API_"):
    config['carto'] = {
        'base_url': env.str('CARTO_BASE_URL', ''),
        'user': env.str('CARTO_ADMIN_USER', 'xxx'),
        'api_key': env.str('CARTO_ADMIN_API_KEY', 'xxx'),
    }
    
# Step 2: create carto variable
# create carto object to be able to use methods inside it. We import all necessary classes for the samples too 

from etl.cf_model import CartoFramesModel, GeometryType
from etl.constants.global_constants import meters_in_mile
from cartoframes import to_carto, create_table_from_query, read_carto
from catboost import CatBoostRegressor

carto = CartoFramesModel()

/home/jacrisol/.virtualenvs/vantage/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Inputs and parameters

In [30]:
can_candidate = 'vtg_test_ws_cann_candidate'
can_features = ['o_urbanicity',
                'o_popcy',
                'n_urbanicity',
                'n_popcy',
                'distancemiles']

In [31]:
def bain_method():
    mod = CatBoostRegressor()
    mod.load_model('../../../../models/can_model.cbm')
    df = carto.get_geopandas_from_query(can_candidate)
    X = df[can_features]
    preds = mod.predict(X)
    preds_df = df.copy()
    preds_df['pred_can'] = preds
    
    # if we need to upload pred_Can, is mandatory target table have got this field. In this case, can_candidate
    # need to have got pred_can field. We added this field ni enrich cannibalization notebook.
    response = carto.update_rows(can_candidate, 'cartodb_id', ['pred_can'], preds_df[['cartodb_id', 'pred_can']])

In [33]:
bain_method()

[2021-02-05T18:21:25Z] (273662) {cf_model.py:162} INFO - Processing data
[2021-02-05T18:21:50Z] (273662) {cf_model.py:178} INFO - Updating data ...
[2021-02-05T18:21:55Z] (273662) {cf_model.py:181} INFO - Updated !
